In [ ]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

# Datos relativos dentro del repo
PATH = 'data/resultado_anon.csv'

# Carga y tipificación
df = pd.read_csv(PATH, dtype=str)
df['Fecha_dt'] = pd.to_datetime(df['Fecha'], errors='coerce', dayfirst=True)
df['Año'] = df['Fecha_dt'].dt.year

num_cols = ['Total','1°','2°','Pos.','10+X','X','10s','9s','6','5','11','10']
for c in num_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors='coerce')

for c in ['Disciplina','Categoria','Club','Arquero']:
    if c in df.columns:
        df[c] = df[c].fillna('Sin dato')

print(f'Filas: {len(df):,}  Años: {int(df["Año"].min())}–{int(df["Año"].max())}')


In [ ]:
disc_opts = ['Todos'] + sorted(df['Disciplina'].dropna().unique().tolist())
disc_dd  = widgets.Dropdown(options=disc_opts, value='Todos', description='Disciplina')
anio_rng = widgets.IntRangeSlider(
    value=[int(df['Año'].min()), int(df['Año'].max())],
    min=int(df['Año'].min()), max=int(df['Año'].max()),
    step=1, description='Año', continuous_update=False
)
display(widgets.HBox([disc_dd, anio_rng]))


In [ ]:
def filtered():
    f = df[(df['Año']>=anio_rng.value[0]) & (df['Año']<=anio_rng.value[1])].copy()
    if disc_dd.value!='Todos':
        f = f[f['Disciplina']==disc_dd.value]
    return f

def render(*_):
    f = filtered()
    if f.empty:
        print('Sin datos para el filtro'); return

    # 1) Promedio anual de Total
    g1 = f.groupby('Año')['Total'].mean().reset_index()
    plt.figure(figsize=(9,3.8))
    plt.plot(g1['Año'], g1['Total'], marker='o')
    plt.title('Promedio anual de puntaje')
    plt.xlabel('Año'); plt.ylabel('Promedio Total'); plt.grid(True); plt.tight_layout(); plt.show()

    # 2) Tasa de podio anual
    if 'Pos.' in f.columns:
        f['Podio'] = f['Pos.'].isin([1,2,3]).astype(int)
        g2 = f.groupby('Año').agg(Podios=('Podio','sum'), Participaciones=('Podio','count')).reset_index()
        g2['TasaPodio_%'] = (g2['Podios']/g2['Participaciones']*100).round(2)
        plt.figure(figsize=(9,3.8))
        plt.bar(g2['Año'], g2['TasaPodio_%'])
        plt.title('Tasa de podios por año')
        plt.xlabel('Año'); plt.ylabel('Podios (%)'); plt.tight_layout(); plt.show()

# vincular widgets a render
widgets.interactive_output(render, {'_': disc_dd.observe(lambda c: render(), 'value')});
widgets.interactive_output(render, {'_': anio_rng.observe(lambda c: render(), 'value')});
render()
